In [ ]:
import pandas as pd
import numpy as np
from p_tqdm import p_map

In [ ]:
from const import DATASETS,SETTINGS


### Merge all centroids for relevant Layer. 

In [ ]:
# def get_centroids(kind):
areatype = 'Middle_Layer_Super_Output_Areas'    





## GENERATE CENTROIDS AND NAME ARRAY

In [ ]:
cdf = pd.DataFrame()
for y in SETTINGS.years:
    f = SETTINGS.matchC(areatype,y)
    print(f)
    dummy = pd.read_csv(f)
    dummy['YEAR'] = y
    # dummy.columns = ['AREACD' if 'cd' in x.lower() else x for x in dummy.columns]
    cdf = pd.concat([cdf, dummy ],axis=0)# df.append(process_df(url,src))


# cdf = pd.read_csv('/Users/danielellis/ONSDataServer/processing/centroids/national_archives/Lower_layer_super_output_areas_(E+W)_2011_Population_Weighted_Centroids_V2/LSOA_2011_EW_PWC_COORD_V2.CSV')[['LSOA11CD','LONGITUDE','LATITUDE']]
# print(cdf)
# # [['LSOA11CD','LONGITUDE','LATITUDE']]
    
    # '/Users/danielellis/ONSDataServer/processing/centroids/national_archives/Middle_layer_super_output_areas_(E+W)_2011_Population_Weighted_Centroids_V2/MSOA_2011_EW_PWC_COORD_V2.CSV')[['MSOA11CD','LONGITUDE','LATITUDE']]
cdf['YEAR'] = 2011

cdf.columns = 'AREACD LAT LON YEAR'.split()
cdf.groupby('AREACD').mean().reset_index()
cdf['GEOGRAPHY_CODE'] = cdf.index
cdf = cdf.astype({'AREACD': str,
            'LAT': np.int32,
            'LON': np.int32,
            'YEAR': np.int32,
            'GEOGRAPHY_CODE': np.int32})

geomap = dict(zip(cdf.AREACD,cdf.GEOGRAPHY_CODE))
cdf

##  A function to get the individual dataframes

In [ ]:
def process_df(url, src):
    '''Read each individaul query and return a dataframe'''
    print(url)
    dummydf = pd.read_csv(url)
    dummydf.columns = 'GEOGRAPHY_CODE YEAR CELL OBS_VALUE'.split()

    if 'cellmaps' in src:
        cells = '0,'+','.join(src['cellnames'])
        cells = ','.join(src['cellnames'])
        for v, k in src['cellmaps'].items(): 
            for c in k: 
                cells = cells.replace(c,v)
        src['cellnames'] = ('Totals,'+cells).split(',')

    mapping = dict(zip(eval(src['cells']), src['cellnames']))
    dummydf['CELL'] = dummydf['CELL'].apply(lambda x: mapping[x])   # (rename,mapping = mapping)
    return dummydf



## Process each Year and concat

In [ ]:
def fetch(dataset):
    '''Fetch the data from the source'''
    global SETTINGS
    sources = dataset['sources']
    #     src = sources[0].copy()
    #      dont use pointers copy

    df = False
    
    for src in sources:
        src = src.copy()
        src['cells'] = ','.join(map(str, src['cells']))
        src['selection'] = ','.join(map(str, src['selection']))
        src['key'] = SETTINGS.conf['apikey']
        src['geocode'] = SETTINGS.geocode[areatype][src['year']]
        if not hasattr(src, 'query_misc'): src['query_misc'] = ''

        url = SETTINGS.conf['apiurl'].format(**src)
        print(src)
        if type(df) != bool: df = pd.concat([df, process_df(url, src)]) # df.append(process_df(url,src))
        else: df = process_df(url, src)
        print(df.head())

    # return df
    print('mapping')
    # subcategory = mapping
    df = df[['GEOGRAPHY_CODE', 'CELL', 'OBS_VALUE','YEAR']]
    df = df.sort_values('CELL')
    order = df.CELL.drop_duplicates().to_list()


    if 'Totals' not in order: 
        b = df.groupby('GEOGRAPHY_CODE').sum()
        b.CELL = 'Totals'
        df = pd.concat([df,b.reset_index()],axis=0)  
        order=['Totals']+order

    mapping = dict(zip(order, range(len(order))))
    

    # datamong
    df['GEOGRAPHY_CODE'] = df['GEOGRAPHY_CODE'].apply(lambda x: geomap[x]).astype(np.uint16)
    i = df.groupby(['GEOGRAPHY_CODE', 'CELL']).sum().astype(np.uint32).reset_index()
    i.CELL = [mapping[k] for k in i.CELL]


    # return i[['GEOGRAPHY_CODE', 'CELL', 'OBS_VALUE']]]
    

    return [dataset['code'], pd.DataFrame(order,columns = ['CELL','name']).set_index('CELL') , i[['GEOGRAPHY_CODE', 'CELL', 'OBS_VALUE']]]


In [ ]:
SETTINGS.geocode[areatype]

### Parallel Fetch 


In [ ]:

results = p_map(fetch,DATASETS)

# results = []
# for i in DATASETS:
#     results.append(fetch(i))

# print(results)

# a = fetch(DATASETS[4])

In [ ]:
a = results[1]
print(a)
print(a[a.YEAR==2011])
# [geomap[i] for i in a.GEOGRAPHY_CODE]

geomap
e=a[:].GEOGRAPHY_CODE.drop_duplicates()
print(e)
s = 0 
f = 0
miss = []
for i in e.values:

    try:
        (geomap[i])
        s+=1
    except:
        f+=1
        miss.append(i)
print(s,f,len(miss))
        
print(miss[:20])
geomap

In [ ]:

db_file = f'./{areatype}.sqlite'

import sqlite3
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))
conn = sqlite3.connect(db_file)
# cur = conn.cursor()


for name,order,df in results:
    print(name)
    df.to_sql(name, con=conn, if_exists='replace',index_label='GEOMETRY_ID')
    order.to_sql(name+'_names', con=conn, if_exists='replace',index_label='CELL')



### Write To DB

In [ ]:
SETTINGS.centroids

In [ ]:
a[a.YEAR==2011]

### SHRINK THE DATABASE

In [ ]:
from pathlib import Path
file_size = Path(db_file).stat().st_size

print("The file size WAS:", file_size/1048576,"MB")

conn.execute("VACUUM") ### SHRINK!!!!
conn.close()

file_size = Path(db_file).stat().st_size

print("The file size is:", file_size/1048576,"MB")

### CENTROIDS table